## Import libraries 📚

In [2]:
import numpy as np
from tqdm.notebook import tqdm

In [4]:
# run it twice if it doesn't work
from tsai.all import *
computer_setup()

os             : Linux-5.4.0-80-generic-x86_64-with-debian-bullseye-sid
python         : 3.7.3
tsai           : 0.2.22
fastai         : 2.5.2
fastcore       : 1.3.26
torch          : 1.9.1+cu102
n_cpus         : 10
device         : cuda (Tesla T4)


## How to use the original ROCKET method? 🚀

ROCKET is applied in 2 phases:

1. Generate features from each time series: ROCKET calculates 20k features from each time series, independently of the sequence length. 
2. Apply a classifier to those calculated features. Those features are then used by the classifier of your choice. In the original code they use 2 simple linear classifiers: RidgeClassifierCV and Logistic Regression, but you can use any classifier.

### 1️⃣ Generate features

Let's first generate the features. We'll import data from a UCR Time Series dataset.

The original method requires the time series to be in a 2d array of shape (samples, len). Remember than only univariate sequences are allow in this original method.

In [ ]:
time = 256
data_p1 = np.load("../../Pre-Processing/trials/subject_1_session_1_filt_ica_car.npy")
data_p2 = np.load("../../Pre-Processing/trials/subject_1_session_2_filt_ica_car.npy")

data_s1 = np.concatenate((data_p1, data_p2), axis = 0)
labels_s1 = np.array([0,1,2,3]*10)

data_s1 = data_s1[:,:,:time]

data_p1_2 = np.load("../../Pre-Processing/trials/subject_2_session_1_filt_ica_car.npy")
data_p2_2 = np.load("../../Pre-Processing/trials/subject_2_session_2_filt_ica_car.npy")

data_s2 = np.concatenate((data_p1_2, data_p2_2), axis = 0)
labels_s2 = np.array([0,1,2,3]*10)

data_s2 = data_s2[:,:,:time]

## How to use ROCKET with large and/ or multivariate datasets on GPU? - Recommended ⭐️

As stated before, the current ROCKET method doesn't support multivariate time series or GPU. This may be a drawback in some cases. 

To overcome both limitations I've created a multivariate ROCKET on GPU in Pytorch. 

### 1️⃣ Generate features

First you prepare the input data and normalize it per sample. The input to ROCKET Pytorch is a 3d tensor of shape (samples, vars, len), preferrable on gpu.

The way to use ROCKET in Pytorch is the following:

* Create a dataset as you would normally do in `tsai`. 
* Create a TSDataLoaders with the following kwargs: 
    * drop_last=False. In this way we get features for every input sample.
    * shuffle_train=False
    * batch_tfms=[TSStandardize(by_sample=True)] so that input is normalized by sample, as recommended by the authors


In [ ]:
#X, y, splits = get_UCR_data('HandMovementDirection', split_data=False)
#splits = RandomSplitter()(range_of(data_p1))

#data_p1 = np.load("../Pre-Processing/trials/subject_2_session_1_filt_ica_car.npy")
#labels_p1 = np.array([0,1,2, 3]*5)

#data_p2 = np.load("../Pre-Processing/trials/subject_2_session_2_filt_ica_car.npy")
#labels_p2 = np.array([0,1,2, 3]*5)

splits_1 = TrainValidTestSplitter(stratify = True, random_state= 10, valid_size = 0.)(range_of(data_s1))
splits_2 = TrainValidTestSplitter(stratify = True, random_state= 0, valid_size = 0.)(range_of(data_s2))

splits = (splits_1[0], splits_2[0])
for i in range(0,40):
    splits[1][i] += 40

In [ ]:
data_s12 = np.concatenate((data_s1, data_s2), axis = 0)
labels_s12 = np.array([0,1,2,3]*20)
# np.random.shuffle(labels_s12) #shuffling to see whether the model is learning or not 
data_s12 = data_s12[:,:,:time]

In [ ]:
tfms  = [None, [Categorize()]]
batch_tfms = [TSStandardize(by_sample=True)]
dls = get_ts_dls(data_s12, labels_s12,splits = splits, tfms=tfms, drop_last=False, 
                 shuffle_train=True, batch_tfms=batch_tfms, bs=10_000)

☣️☣️ You will be able to create a dls (TSDataLoaders) object with unusually large batch sizes. I've tested it with a large dataset and a batch size = 100_000 and it worked fine. This is because ROCKET is not a usual Deep Learning model. It just applies convolutions (kernels) one at a time to create the features.

Instantiate a rocket model with the desired n_kernels (authors use 10_000) and kernel sizes (7, 9 and 11 in the original paper). 

In [ ]:
model = build_ts_model(ROCKET, dls=dls, n_kernels = 20000, kss = [7, 9, 11]) # n_kernels=10_000, kss=[7, 9, 11] set by default, but you can pass other values as kwargs

Now generate rocket features for the entire train and valid datasets using the create_rocket_features convenience function `create_rocket_features`.

And we now transform the original data, creating 20k features per sample

In [ ]:
X_train, y_train = create_rocket_features(dls.train, model)
X_valid, y_valid = create_rocket_features(dls.valid, model)
X_train.shape, X_valid.shape

### 2️⃣ Apply a classifier

Once you build the 20k features per sample, you can use them to train any classifier of your choice.

#### RidgeClassifierCV

And now you apply a classifier of your choice. 
With RidgeClassifierCV in particular, there's no need to normalize the calculated features before passing them to the classifier, as it does it internally (if normalize is set to True as recommended by the authors).

In [ ]:
from sklearn.linear_model import RidgeClassifierCV
ridge = RidgeClassifierCV(alphas=np.logspace(-8, 8, 17), normalize=True)
ridge.fit(X_train, y_train)
print(f'alpha: {ridge.alpha_:.2E}  train: {ridge.score(X_train, y_train):.5f}  valid: {ridge.score(X_valid, y_valid):.5f}')

## Averaging over 50 Seeds

In [22]:
valid_scores = []
time = 256
n_kernels = 2000
    
for i in tqdm(range(0, 50)):
    
    data_p1 = np.load("../../Pre-Processing/trials/subject_1_session_1_filt_ica_car.npy")
    data_p2 = np.load("../../Pre-Processing/trials/subject_1_session_2_filt_ica_car.npy")
    data_s1 = np.concatenate((data_p1, data_p2), axis = 0)
    data_s1 = data_s1[:,:,:time]

    data_p1_2 = np.load("../../Pre-Processing/trials/subject_2_session_1_filt_ica_car.npy")
    data_p2_2 = np.load("../../Pre-Processing/trials/subject_2_session_2_filt_ica_car.npy")
    data_s2 = np.concatenate((data_p1_2, data_p2_2), axis = 0)
    data_s2 = data_s2[:,:,:time]
    
    splits_1 = TrainValidTestSplitter(stratify = True, random_state= 1000 -i, valid_size = 0.)(range_of(data_s1))
    splits_2 = TrainValidTestSplitter(stratify = True, random_state= 0 + i, valid_size = 0.)(range_of(data_s2))

    splits = (splits_1[0], splits_2[0])
    for i in range(0,40):
        splits[1][i] += 40
    
    data_s12 = np.concatenate((data_s1, data_s2), axis = 0)
    labels_s12 = np.array([0,1,2,3]*20)
    data_s12 = data_s12[:,:,:time]
    
    tfms  = [None, [Categorize()]]
    batch_tfms = [TSStandardize(by_sample=True)]
    dls = get_ts_dls(data_s12, labels_s12,splits = splits, tfms=tfms, drop_last=False, 
                     shuffle_train=True, batch_tfms=batch_tfms, bs=10_000)
    
    model = build_ts_model(ROCKET, dls=dls, n_kernels = n_kernels, kss = [7, 9, 11]) 
    # n_kernels=10_000, kss=[7, 9, 11] set by default, but you can pass other values as kwargs
    
    X_train, y_train = create_rocket_features(dls.train, model)
    X_valid, y_valid = create_rocket_features(dls.valid, model)
    
    ridge = RidgeClassifierCV(alphas=np.logspace(-8, 8, 17), normalize=True)
    ridge.fit(X_train, y_train)
    valid_scores.append(ridge.score(X_valid, y_valid))


  0%|          | 0/50 [00:00<?, ?it/s]

In [23]:
print(sum(valid_scores)/len(valid_scores), np.std(valid_scores))

1.0 0.0
